<a href="https://colab.research.google.com/github/Anshad-Aziz/Machine_Learning/blob/main/Predict_Discount.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn import linear_model
from sklearn.metrics import r2_score
import plotly.express as px
from sklearn.tree import DecisionTreeRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error,r2_score
from sklearn.model_selection import cross_val_score

In [2]:
train=pd.read_csv('train.csv')

In [3]:
train.head()

,id,title,Rating,maincateg,platform,price1,actprice1,Offer %,norating1,noreviews1,star_5f,star_4f,star_3f,star_2f,star_1f,fulfilled1
0,16695,Fashionable & Comfortable Bellies For Women (...,3.9,Women,Flipkart,698,999,30.13%,38.0,7.0,17.0,9.0,6.0,3,3,0
1,5120,Combo Pack of 4 Casual Shoes Sneakers For Men ...,3.8,Men,Flipkart,999,1999,50.03%,531.0,69.0,264.0,92.0,73.0,29,73,1
2,18391,Cilia Mode Leo Sneakers For Women (White),4.4,Women,Flipkart,2749,4999,45.01%,17.0,4.0,11.0,3.0,2.0,1,0,1
3,495,Men Black Sports Sandal,4.2,Men,Flipkart,518,724,15.85%,46413.0,6229.0,1045.0,12416.0,5352.0,701,4595,1
4,16408,Men Green Sports Sandal,3.9,Men,Flipkart,1379,2299,40.02%,77.0,3.0,35.0,21.0,7.0,7,7,1


In [4]:
train.shape

(15730, 16)

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15730 entries, 0 to 15729
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          15730 non-null  int64  
 1   title       15730 non-null  object 
 2   Rating      15730 non-null  float64
 3   maincateg   15204 non-null  object 
 4   platform    15730 non-null  object 
 5   price1      15730 non-null  int64  
 6   actprice1   15730 non-null  int64  
 7   Offer %     15730 non-null  object 
 8   norating1   15052 non-null  float64
 9   noreviews1  15152 non-null  float64
 10  star_5f     15142 non-null  float64
 11  star_4f     15191 non-null  float64
 12  star_3f     15499 non-null  float64
 13  star_2f     15730 non-null  int64  
 14  star_1f     15730 non-null  int64  
 15  fulfilled1  15730 non-null  int64  
dtypes: float64(6), int64(6), object(4)
memory usage: 1.9+ MB


In [6]:
train.describe()

,id,Rating,price1,actprice1,norating1,noreviews1,star_5f,star_4f,star_3f,star_2f,star_1f,fulfilled1
count,15730.000000,15730.000000,15730.000000,15730.000000,15052.000000,15152.000000,15142.000000,15191.000000,15499.000000,15730.000000,15730.000000,15730.000000
mean,10479.541577,4.012873,688.070693,1369.286777,3057.660776,423.976307,1585.239466,655.923310,357.260662,155.085188,275.500572,0.601526
std,6080.166276,0.298440,649.409586,1240.900227,11846.965689,1768.230384,6177.476241,2855.735531,1402.246610,558.650254,958.589075,0.489600
min,3.000000,0.000000,69.000000,42.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5212.000000,3.900000,349.000000,699.000000,63.000000,9.000000,30.000000,12.000000,7.000000,3.000000,6.000000,0.000000
50%,10458.500000,4.000000,474.000000,999.000000,308.000000,44.000000,150.000000,60.000000,34.000000,17.000000,30.000000,1.000000
75%,15766.750000,4.200000,699.000000,1299.000000,1526.000000,215.000000,788.000000,300.000000,172.000000,77.000000,140.000000,1.000000
max,20973.000000,5.000000,5998.000000,13499.000000,289973.000000,45448.000000,151193.000000,74037.000000,34978.000000,11705.000000,18060.000000,1.000000


In [7]:
train['Offer %']=train['Offer %'].str[0:4]
train.head()

,id,title,Rating,maincateg,platform,price1,actprice1,Offer %,norating1,noreviews1,star_5f,star_4f,star_3f,star_2f,star_1f,fulfilled1
0,16695,Fashionable & Comfortable Bellies For Women (...,3.9,Women,Flipkart,698,999,30.1,38.0,7.0,17.0,9.0,6.0,3,3,0
1,5120,Combo Pack of 4 Casual Shoes Sneakers For Men ...,3.8,Men,Flipkart,999,1999,50.0,531.0,69.0,264.0,92.0,73.0,29,73,1
2,18391,Cilia Mode Leo Sneakers For Women (White),4.4,Women,Flipkart,2749,4999,45.0,17.0,4.0,11.0,3.0,2.0,1,0,1
3,495,Men Black Sports Sandal,4.2,Men,Flipkart,518,724,15.8,46413.0,6229.0,1045.0,12416.0,5352.0,701,4595,1
4,16408,Men Green Sports Sandal,3.9,Men,Flipkart,1379,2299,40.0,77.0,3.0,35.0,21.0,7.0,7,7,1


In [8]:
train['Offer %']=train['Offer %'].astype('float')
train.head()

,id,title,Rating,maincateg,platform,price1,actprice1,Offer %,norating1,noreviews1,star_5f,star_4f,star_3f,star_2f,star_1f,fulfilled1
0,16695,Fashionable & Comfortable Bellies For Women (...,3.9,Women,Flipkart,698,999,30.1,38.0,7.0,17.0,9.0,6.0,3,3,0
1,5120,Combo Pack of 4 Casual Shoes Sneakers For Men ...,3.8,Men,Flipkart,999,1999,50.0,531.0,69.0,264.0,92.0,73.0,29,73,1
2,18391,Cilia Mode Leo Sneakers For Women (White),4.4,Women,Flipkart,2749,4999,45.0,17.0,4.0,11.0,3.0,2.0,1,0,1
3,495,Men Black Sports Sandal,4.2,Men,Flipkart,518,724,15.8,46413.0,6229.0,1045.0,12416.0,5352.0,701,4595,1
4,16408,Men Green Sports Sandal,3.9,Men,Flipkart,1379,2299,40.0,77.0,3.0,35.0,21.0,7.0,7,7,1


In [10]:
train['norating1']=train['norating1'].fillna(train['norating1'].sum())

In [11]:
train['noreviews1']=train['noreviews1'].fillna(train['noreviews1'].sum())


In [12]:
total_cus=train['norating1']+train['star_5f']+train['star_4f']+train['star_3f']+train['star_2f']+train['star_1f']

In [13]:
train['total_cus']=total_cus

In [14]:
train=train.drop(['star_5f','star_4f','star_3f','star_2f','star_1f'],axis=1)
train.shape

(15730, 12)

In [15]:
train=train.drop(['norating1','noreviews1'],axis=1)

In [16]:
train['platform']=train['platform'].map({'Flipkart':1,'Amazon':0})

In [17]:
train.loc[train['title'].str.upper().str.contains('WOMEN')&train['maincateg'].isnull(),'maincateg']='Women'
train.loc[train['title'].str.upper().str.contains('MEN')&train['maincateg'].isnull(),'maincateg']='Men'
train['maincateg'].isnull().sum()

57

In [18]:
train.loc[train['maincateg'].isnull(),'maincateg']='Unisex'
print(train['maincateg'].isnull().sum())
print(train['maincateg'].unique())

0
['Women' 'Men' 'Unisex']


In [19]:
train.head()

,id,title,Rating,maincateg,platform,price1,actprice1,Offer %,fulfilled1,total_cus
0,16695,Fashionable & Comfortable Bellies For Women (...,3.9,Women,1,698,999,30.1,0,76.0
1,5120,Combo Pack of 4 Casual Shoes Sneakers For Men ...,3.8,Men,1,999,1999,50.0,1,1062.0
2,18391,Cilia Mode Leo Sneakers For Women (White),4.4,Women,1,2749,4999,45.0,1,34.0
3,495,Men Black Sports Sandal,4.2,Men,1,518,724,15.8,1,70522.0
4,16408,Men Green Sports Sandal,3.9,Men,1,1379,2299,40.0,1,154.0


In [20]:
train=pd.get_dummies(train,columns=['maincateg'])

In [21]:
train.head()

,id,title,Rating,platform,price1,actprice1,Offer %,fulfilled1,total_cus,maincateg_Men,maincateg_Unisex,maincateg_Women
0,16695,Fashionable & Comfortable Bellies For Women (...,3.9,1,698,999,30.1,0,76.0,0,0,1
1,5120,Combo Pack of 4 Casual Shoes Sneakers For Men ...,3.8,1,999,1999,50.0,1,1062.0,1,0,0
2,18391,Cilia Mode Leo Sneakers For Women (White),4.4,1,2749,4999,45.0,1,34.0,0,0,1
3,495,Men Black Sports Sandal,4.2,1,518,724,15.8,1,70522.0,1,0,0
4,16408,Men Green Sports Sandal,3.9,1,1379,2299,40.0,1,154.0,1,0,0


In [22]:
from collections import Counter
arr=[]
for i in train['title'].str.split(' '):
   arr+=i

c=Counter(arr)
c_list=c.most_common(100)
c_list

[('For', 10241),
 ('Sandal', 4194),
 ('Shoes', 4163),
 ('Women', 3976),
 ('Running', 2024),
 ('Sneakers', 1806),
 ('Men', 1580),
 ('Black', 1499),
 ('Boots', 1471),
 ('Casual', 1454),
 ('Men\xa0\xa0(Black)', 1454),
 ('Bellies', 1093),
 ('for', 1080),
 ('Jutis', 1073),
 ('Flats', 1054),
 ('Heels', 1008),
 ('Women\xa0\xa0(Black)', 967),
 ('Flip', 894),
 ('Wedges', 875),
 ('Flops', 843),
 ('Stylish', 830),
 ('Walking', 775),
 ('Slip', 711),
 ('Slippers', 679),
 ('On', 663),
 ('Sports', 653),
 ('&', 639),
 ('Slides', 618),
 ('Casuals', 614),
 ('shoes', 593),
 ('Girls', 592),
 ('Women\xa0\xa0(Pink)', 578),
 ('Men\xa0\xa0(Brown)', 529),
 ('|', 505),
 ('Leather', 469),
 ('of', 458),
 ('Mojaris', 458),
 ('Pink', 455),
 ("Men's", 445),
 ('Women\xa0\xa0(Multicolor)', 442),
 ('Lace', 435),
 ('Brown', 435),
 ('Women\xa0\xa0(White)', 432),
 ('and', 430),
 ('Combo', 403),
 ('Grey', 378),
 ('Blue', 371),
 ('Pack', 361),
 ('Women\xa0\xa0(Grey)', 360),
 ('Men\xa0\xa0(Blue)', 356),
 ('High', 351),
 ('La

In [23]:
train['combo']=train['title'].str.upper().str.contains('COMBO').astype(int)

In [24]:
sp_cols=['Running','Black', 'Boots', 'Casual', 'Bellies', 'Jutis', 'Flats', 'Heels', 'Wedges', 'Stylish', 'Walking', 'Slip', 'Sports', 'Girls', 'Leather', 'Mojaris', 'Pink', 'Lace', 'Brown', 'Grey', 'Blue', 'Pack', 'Loafers', 'Beige', 'White', 'Red', 'Ethnic', 'Multicolor', 'Formal', 'Tan', 'Synthetic', 'Gold', 'Canvas', 'Party', 'Training', 'IDP', 'Gym', 'Jutti', 'Ankle', 'New', 'Navy', 'Lightweight', 'Derby']

In [25]:
train['sandal']=train['title'].str.upper().str.contains('SANDAL').astype(int)
train['sneaker']=train['title'].str.upper().str.contains('SNEAKER').astype(int)
train['shoe']=train['title'].str.upper().str.contains('SHOE').astype(int)
train['chappal'] = (train['title'].str.upper().str.contains('SLIPPERS') | train['title'].str.upper().str.contains('SLIDES') | train['title'].str.upper().str.contains('FLIP FLOPS')).astype(int)

In [26]:
train.head()

,id,title,Rating,platform,price1,actprice1,Offer %,fulfilled1,total_cus,maincateg_Men,maincateg_Unisex,maincateg_Women,combo,sandal,sneaker,shoe,chappal
0,16695,Fashionable & Comfortable Bellies For Women (...,3.9,1,698,999,30.1,0,76.0,0,0,1,0,0,0,0,0
1,5120,Combo Pack of 4 Casual Shoes Sneakers For Men ...,3.8,1,999,1999,50.0,1,1062.0,1,0,0,1,0,1,1,0
2,18391,Cilia Mode Leo Sneakers For Women (White),4.4,1,2749,4999,45.0,1,34.0,0,0,1,0,0,1,0,0
3,495,Men Black Sports Sandal,4.2,1,518,724,15.8,1,70522.0,1,0,0,0,1,0,0,0
4,16408,Men Green Sports Sandal,3.9,1,1379,2299,40.0,1,154.0,1,0,0,0,1,0,0,0


In [27]:
for cl in sp_cols:
    train[cl] = train['title'].str.upper().str.contains(cl.upper()).astype(int)

In [28]:
train.drop('title', axis='columns', inplace=True)

In [29]:
train = train[(['Rating','actprice1', 'price1', 'fulfilled1', 'sandal', 'sneaker', 'shoe', 'chappal', 'maincateg_Men','maincateg_Women', 'platform','combo']) + sp_cols]
train.head()

,Rating,actprice1,price1,fulfilled1,sandal,sneaker,shoe,chappal,maincateg_Men,maincateg_Women,platform,combo,Running,Black,Boots,Casual,Bellies,Jutis,Flats,Heels,Wedges,Stylish,Walking,Slip,Sports,Girls,Leather,Mojaris,Pink,Lace,Brown,Grey,Blue,Pack,Loafers,Beige,White,Red,Ethnic,Multicolor,Formal,Tan,Synthetic,Gold,Canvas,Party,Training,IDP,Gym,Jutti,Ankle,New,Navy,Lightweight,Derby
0,3.9,999,698,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,3.8,1999,999,1,0,1,1,0,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,4.4,4999,2749,1,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4.2,724,518,1,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,3.9,2299,1379,1,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [31]:
x=train.drop(['price1'],axis=1)
y=train['price1']

In [32]:
x.shape,y.shape

((15730, 54), (15730,))

In [33]:
dt=DecisionTreeRegressor()
score=cross_val_score(dt,x,y,cv=5).mean()
score

0.8664117137370011

In [34]:
lgb=LGBMRegressor()
score=cross_val_score(lgb,x,y).mean()
score

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005549 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 353
[LightGBM] [Info] Number of data points in the train set: 12584, number of used features: 54
[LightGBM] [Info] Start training from score 691.079704
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003640 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 356
[LightGBM] [Info] Number of data points in the train set: 12584, number of used features: 54
[LightGBM] [Info] Start training from score 685.169263
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005766 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory i

0.8798638478762479

In [35]:
xgb=XGBRegressor()
score=cross_val_score(xgb,x,y).mean()
score

0.8995040938712087